In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import knn
import random

In [46]:
separacion = 50
clusterNum = int(2000/4)

data = [[1, random.random() * separacion + 40, random.random() * separacion + 30] for i in range(clusterNum)]
data += [[2, random.random() * separacion + 0, random.random() * separacion + 30] for i in range(clusterNum)]
data += [[3, random.random() * separacion + 20, random.random() * separacion + -10] for i in range(clusterNum)]
data += [[4, random.random() * separacion + -5, random.random() * separacion + 80] for i in range(clusterNum)]
random.shuffle(data)
df = pd.DataFrame(np.array(data), columns=["Class", "A", "B"])
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Class   2000 non-null   float64
 1   A       2000 non-null   float64
 2   B       2000 non-null   float64
dtypes: float64(3)
memory usage: 47.0 KB


In [47]:
# Se sustituyen valores categóricos por valores numéricos

dfNum = pd.DataFrame()
for col, t in zip(df.columns, df.dtypes):
    pos = 0
    keys = pd.DataFrame()    
    if (t == 'object'):
        key = dict([(k,i) for i,k in enumerate(df.loc[:, col].unique())])
        dfNum[col] = df[col].map(key)
    else:
        dfNum[col] = df[col]

,Class,A,B
0,1.0,80.824052,41.402769
1,3.0,32.934855,8.690374
2,3.0,27.305562,19.017937
3,3.0,40.667776,27.794964
4,4.0,14.969836,126.566896
...,...,...,...
1995,2.0,38.708068,46.154017
1996,4.0,35.641669,113.538001
1997,2.0,48.833250,43.847999
1998,3.0,42.490052,7.445627


In [48]:
# Se filtran las columnas redundantes
r = 0.0
corrMat = dfNum.corr()
relevantCols = corrMat.loc[(corrMat['Class'] > r) | (corrMat['Class'] < -r)].transpose().columns
dfNum = dfNum.loc[:, relevantCols.to_list()]

In [49]:
# Se separan la columna a predecir de los atributos

individuals = pd.DataFrame(dfNum.iloc[:,1:])
individuals

,A,B
0,80.824052,41.402769
1,32.934855,8.690374
2,27.305562,19.017937
3,40.667776,27.794964
4,14.969836,126.566896
...,...,...
1995,38.708068,46.154017
1996,35.641669,113.538001
1997,48.833250,43.847999
1998,42.490052,7.445627


In [50]:
target = pd.DataFrame(dfNum['Class'])
target

,Class
0,1.0
1,3.0
2,3.0
3,3.0
4,4.0
...,...
1995,2.0
1996,4.0
1997,2.0
1998,3.0


In [51]:
# Normalización
individuals = pd.DataFrame((individuals - individuals.min())/(individuals.max() - individuals.min()))
individuals

,A,B
0,0.906136,0.366844
1,0.399781,0.133045
2,0.340260,0.206857
3,0.481545,0.269587
4,0.209828,0.975519
...,...,...
1995,0.460824,0.400801
1996,0.428401,0.882401
1997,0.567882,0.384320
1998,0.500813,0.124149


In [52]:
# Se separa el conjunto de datos en entrenamiento y test

xTrain, xTest, yTrain, yTest = train_test_split(individuals, target, test_size=0.3)
classes = dfNum['Class'].unique()
classes


array([1., 3., 4., 2.])

In [53]:
# Se instancia el clasificador y se añaden los datos

classifier = knn.Knn(100)
classifier.fit(xTrain, yTrain)
# classifier.predSingle(yTrain[0])

In [54]:
# classifier.predSingle(xTrain.iloc[1])

In [55]:
predicted = classifier.pred(xTest)
precision, confusionMatrix = classifier.precision(predicted, yTest, classes)
confusionMatrix = pd.DataFrame(confusionMatrix, columns=np.concatenate((np.array(["Class"]), classes)))
print("Fila = predicho, Columna = real")
print("Precisión = " + str(precision))
confusionMatrix


Fila = predicho, Columna = real
Precisión = 0.8916666666666667


,Class,1.0,3.0,4.0,2.0
0,1.0,118.0,9.0,0.0,13.0
1,3.0,10.0,138.0,0.0,8.0
2,4.0,0.0,0.0,146.0,2.0
3,2.0,14.0,9.0,0.0,133.0
